In [ ]:
# enter your credentials files path here
creds_file_path = "/home/jovyan/client_secret_1064575295729-8ncp8e1gi66q3rnifh3q8mlgte3rmre2.apps.googleusercontent.com.json"

In [ ]:
!pip3 install google-auth google-api-python-client google-auth-oauthlib

In [ ]:
import io
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Load existing credentials if available
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# If credentials are missing or expired, initiate OAuth flow
if not creds or not creds.valid:
    flow = InstalledAppFlow.from_client_secrets_file(
        creds_file_path,
        SCOPES,
        redirect_uri='https://gpu-notebooks.e2enetworks.com/oauth2callback/'
    )
    authorization_url, _ = flow.authorization_url(prompt='consent')
    print(f'Please go to this URL: {authorization_url}')

    authorization_code = input('Enter the authorization code: ')
    flow.fetch_token(code=authorization_code)
    creds = flow.credentials

    # Save the updated credentials for future use
    with open("token.json", "w") as token:
        token.write(creds.to_json())

# Build the Drive API service
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
#FILE
# Define your query to search for a specific file by name
file_name = "End to Start - Hyderabad 720P.mp4"  # Replace with your actual file name
query = f"name = '{file_name}'"

# Execute the query to search for the file
results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()
items = results.get("files", [])

# Check if any files matching the query were found
if not items:
    print(f'No file named "{file_name}" found.')
else:
    print(f'Files named "{file_name}":')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")


In [ ]:
#FILE
# Define the file ID you want to retrieve
file_id = '1XRPLomaBvaNcd8OU_tUcfPAUbtgw2iNb'  # Replace with the actual file ID

# Fetch metadata for the specified file ID
file = drive_service.files().get(fileId=file_id, fields="id, name, mimeType").execute()

# Check if the file exists
if file:
    print(f"File found:")
    print(f"{file['name']} ({file['id']}) - {file['mimeType']}")
else:
    print(f'File with ID "{file_id}" not found.')


In [ ]:
file_id = '1XRPLomaBvaNcd8OU_tUcfPAUbtgw2iNb'
file_path = '/home/jovyan/End-start-hyd.mp4'

request = drive_service.files().get_media(fileId=file_id)
fh = io.FileIO(file_path, mode='wb')
downloader = MediaIoBaseDownload(fh, request)
done = False

while not done:
    status, done = downloader.next_chunk()

In [ ]:
import subprocess

def crop_video_from_left(input_path, output_path, crop_width):
    try:
        # Define the ffmpeg crop command
        command = [
            'ffmpeg',
            '-i', input_path,  # Input file
            '-vf', f'crop=iw-{crop_width}:ih:{crop_width}:0',  # Crop filter
            '-c:a', 'copy',  # Copy audio
            output_path  # Output file
        ]

        # Run the command
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Check for errors
        if result.returncode != 0:
            print("FFmpeg error:", result.stderr.decode())
        else:
            print(f"Video successfully cropped and saved to: {output_path}")

    except Exception as e:
        print(f"Error: {e}")

# Define the input and output paths and the crop width
input_video_path = "/home/jovyan/End-start-hyd.mp4"
output_video_path = "/home/jovyan/Cropped_End-start-hyd.mp4"
crop_width = 500  # width to crop from the left

# Crop and save the video
crop_video_from_left(input_video_path, output_video_path, crop_width)



In [ ]:
!pip install ultralytics cvzone opencv-python-headless

In [ ]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
import csv
from datetime import datetime
import numpy as np  # Add this import for np.linalg.norm

class Tracker:
    def __init__(self):
        self.center_points = {}
        self.id_count = 0

    def update(self, objects_rect):
        objects_bbs_ids = []

        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + w) // 2
            cy = (y + h) // 2

            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = np.linalg.norm(np.array([cx, cy]) - np.array(pt))

                if dist < 50:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        new_center_points = {obj_bb_id[4]: self.center_points[obj_bb_id[4]] for obj_bb_id in objects_bbs_ids}

        self.center_points = new_center_points.copy()
        return objects_bbs_ids

# Load YOLO model
model = YOLO('/home/jovyan/plantations_best.pt')

# Get class names from the model
class_list = model.names

# Path to the input video
video_path = '/home/jovyan/Cropped_Start-end-hyd-part2.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize tracker
tracker = Tracker()

# Vertical line positions
cx1 = int(0.25 * 1020) + 900
cx2 = cx1 + 16  # distance between the lines
offset = 8

# Create directory for screenshots
screenshot_dir = '/home/jovyan/plantation_sc'
os.makedirs(screenshot_dir, exist_ok=True)

# Path to save the output video in Google Drive
output_video_path = '/home/jovyan/output_Cropped_Start-end-hyd-part2.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (1280, 720))

# Create CSV file and write headers
csv_file = '/home/jovyan/output_Cropped_Start-end-hyd-part2.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Timestamp (00:00:00)", "Label Name", "X", "Y", "Height", "Width"])

# Variables to keep track of crossing objects
crossed_shrub_ids = set()
shrub_count = 0

def process_bbox(bbox_idx):
    global crossed_shrub_ids  # Access the global variable for tracking crossed shrubs
    global shrub_count  # Access the global variable for counting shrubs

    for bbox in bbox_idx:
        x3, y3, x4, y4, id1 = bbox
        cx3 = (x3 + x4) // 2
        cy3 = (y3 + y4) // 2

        # Render bounding boxes
        cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 0, 255), 2)
        cvzone.putTextRect(frame, f'{id1}', (x3, y3), 1, 1)

        # Check if the object crosses the lines and is not counted yet
        if (cx1 < (cx3 + offset) and cx1 > (cx3 - offset)) or (cx2 < (cx3 + offset) and cx2 > (cx3 - offset)):
            if id1 not in crossed_shrub_ids:
                crossed_shrub_ids.add(id1)  # Mark the shrub as counted
                shrub_count += 1  # Increment the count

                # Extract the detected part and save screenshot
                roi = frame[y3:y4, x3:x4]  # Extract the detected part
                cv2.imwrite(f"{screenshot_dir}/shrub_{id1}.png", roi)
                timestamp_s = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Get the timestamp in seconds
                timestamp_00 = str(datetime.fromtimestamp(timestamp_s).strftime('%H:%M:%S'))  # Get the timestamp in 00:00:00 format
                with open(csv_file, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([id1, timestamp_00, 'shrub', x3, y3, y4 - y3, x4 - x3])
    return shrub_count

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1280, 720))

    # Set prediction confidence threshold
    conf_threshold = 0.05
    results = model.predict(frame, conf=conf_threshold)
    a = results[0].boxes.data.cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    list_objects = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])

        if d >= len(class_list):
            print(f"Warning: Class index {d} is out of range for class list with length {len(class_list)}")
            continue

        c = class_list[d]

        if c == 'shrub':  # Only consider the 'shrub' class
            list_objects.append([x1, y1, x2, y2])

    bbox_idx = tracker.update(list_objects)
    shrub_count = process_bbox(bbox_idx)  # Call the function to process bboxes

    # Draw the vertical lines on the frame
    cv2.line(frame, (cx1, 1), (cx1, 500), (255, 255, 255), 1)
    cv2.line(frame, (cx2, 1), (cx2, 500), (255, 255, 255), 1)

    # Display the count of shrubs that crossed the lines
    cv2.putText(frame, f"Shrubs counted: {shrub_count}", (50, 60), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()

# Code to save total counts for each label to a text file
counts = {}
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header
    for row in reader:
        label = row[2]
        if label in counts:
            counts[label] += 1
        else:
            counts[label] = 1

# Save counts to a text file
# count_txt_file = '/content/drive/MyDrive/lines/20_sec_plantation_total_counts.txt'
# with open(count_txt_file, 'w') as file:
#     for label, count in counts.items():
#         file.write(f'{label}: {count}\n')